In [1]:
import numpy as np
import pandas as pd
import psycopg2

#### Let's get from postgres db customer data - active customers in the last 3 years - including order count and calculate cm2%

In [2]:
query = """SELECT cop_c_id, cop_pref_o_ext_s_id, cop_g_order_count, cop_g_net_total, cop_g_cm2
FROM public.customer_order_profile
where cop_g_cm2 is not null
"""

In [3]:
conn = psycopg2.connect("dbname='savings_plan_db' user='dba' host='savings-plan-prod-c-savings-plan-db-readonly.ccsod6gcmvcs.eu-central-1.rds.amazonaws.com' password='m4d_l0y_db4?'")
curs = conn.cursor()
data = pd.read_sql(query, con = conn)
conn.close()

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9404709 entries, 0 to 9404708
Data columns (total 5 columns):
cop_c_id               int64
cop_pref_o_ext_s_id    int64
cop_g_order_count      int64
cop_g_net_total        float64
cop_g_cm2              float64
dtypes: float64(2), int64(3)
memory usage: 358.8 MB


In [5]:
data['CM2%'] = data['cop_g_cm2'] / data['cop_g_net_total'] * 100

In [6]:
data.sample(10)

,cop_c_id,cop_pref_o_ext_s_id,cop_g_order_count,cop_g_net_total,cop_g_cm2,CM2%
8798314,5528020,21,1,51.522064,9.938064,19.288948
7149307,20484093,1,1,20.005800,6.908800,34.533985
7541193,12452194,15,10,994.210564,79.005293,7.946535
1868454,13231500,1,2,309.721311,43.031849,13.893732
7295724,18771611,1,4,215.086764,10.576075,4.917120
2057618,13554664,4,5,229.958333,2.789586,1.213083
8480133,3784739,1,5,101.915920,12.791974,12.551497
6341011,19345385,107,3,102.438016,11.797016,11.516248
2032448,13511481,25,5,463.757389,109.180389,23.542566
8141495,1763255,4,1,34.150000,2.419863,7.085982


In [ ]:
# Get all the existing customers having 2 or more orders finished
# data = data[data['cop_g_order_count'] >= 2]

In [7]:
site_dict = {1: 'Deutschland',
 2: 'International',
 3: 'United Kingdom',
 4: 'France',
 7: 'Netherlands',
 8: 'Poland',
 11: 'Eire',
 12: 'Italy',
 14: 'Belgium',
 15: 'Spain',
 16: 'Czech',
 18: 'Finland',
 19: 'Slovakia',
 20: 'Russia',
 21: 'Denmark',
 22: 'Hungary',
 23: 'Slovenia',
 24: 'Romania',
 25: 'Switzerland',
 26: 'Sweden',
 28: 'Portugal',
 29: 'Croatia',
 30: 'Bulgaria',
 31: 'Norway',
 32: 'Greece'}

In [8]:
np.sort(data['cop_pref_o_ext_s_id'].unique())[:25]

array([ 1,  2,  3,  4,  7,  8, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22, 23,
       24, 25, 26, 28, 29, 30, 31, 32], dtype=int64)

#### Now, from Oracle db, get old and new eligibility status for registered customers that have been active in the last 3 years

In [10]:
import cx_Oracle

In [11]:
query = """select 
k_id,
cz_cti_id old_tier,
cz_invitation_type old_box,
e_tier_id new_tier,
e_invitation_type new_box,
k_registration_s_id
from kunden 
left outer join customers_zooprime on cz_k_id=k_id
left outer join loy_sp.eligibilities on E_EXT_K_ID=k_id
where k_Register = 1
and k_id in (select distinct o_k_id 
from orders
where o_datum > trunc(sysdate - (365 * 3)))
"""

In [12]:
conn = cx_Oracle.connect('readonly', 'read_123', 'db-zpmstb-01:1521/zpmstb.web.zooplus.de', threaded=True, encoding = "UTF-8", nencoding = "UTF-8")
curs = conn.cursor()
o_data = pd.read_sql(query, con = conn)
conn.close()

In [13]:
o_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9613936 entries, 0 to 9613935
Data columns (total 6 columns):
K_ID                   int64
OLD_TIER               float64
OLD_BOX                object
NEW_TIER               float64
NEW_BOX                object
K_REGISTRATION_S_ID    int64
dtypes: float64(2), int64(2), object(2)
memory usage: 440.1+ MB


In [14]:
o_data.head()

,K_ID,OLD_TIER,OLD_BOX,NEW_TIER,NEW_BOX,K_REGISTRATION_S_ID
0,9758541,0.0,Z,0.0,Z,103
1,19611130,0.0,Z,0.0,Z,101
2,14890557,0.0,Z,0.0,Z,18
3,15896569,4.0,A,4.0,A,12
4,12163469,0.0,Z,0.0,Z,21


#### Merging tables by customer ID 

In [78]:
cus_data = o_data.merge(data, how='inner', left_on='K_ID', right_on='cop_c_id')

In [79]:
cus_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9245614 entries, 0 to 9245613
Data columns (total 12 columns):
K_ID                   int64
OLD_TIER               float64
OLD_BOX                object
NEW_TIER               float64
NEW_BOX                object
K_REGISTRATION_S_ID    int64
cop_c_id               int64
cop_pref_o_ext_s_id    int64
cop_g_order_count      int64
cop_g_net_total        float64
cop_g_cm2              float64
CM2%                   float64
dtypes: float64(5), int64(5), object(2)
memory usage: 917.0+ MB


In [80]:
cus_data.sample(5)

,K_ID,OLD_TIER,OLD_BOX,NEW_TIER,NEW_BOX,K_REGISTRATION_S_ID,cop_c_id,cop_pref_o_ext_s_id,cop_g_order_count,cop_g_net_total,cop_g_cm2,CM2%
8368569,14472869,6.0,N,0.0,Z,103,14472869,103,1,43.717057,-2.189963,-5.009402
8016741,2533576,0.0,Z,6.0,N,1,2533576,1,1,46.649571,8.191546,17.559746
8763854,19709698,0.0,Z,0.0,Z,19,19709698,19,2,70.816667,1.670017,2.358226
72441,5913013,4.0,A,4.0,A,7,5913013,7,10,294.330579,32.700219,11.110031
8616004,17940089,6.0,C,0.0,Z,7,17940089,7,1,42.380165,8.758745,20.667086


In [81]:
zp_countries = [ 1,  2,  3,  4,  7,  8, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22, 23,
       24, 25, 26, 28, 29, 30, 31, 32]

In [82]:
cus_data[cus_data['cop_pref_o_ext_s_id'].isin(zp_countries)]['cop_c_id'].count()

8055382

#### Select all customers in Z+ shops and having 2 or more orders (considered as existing customers)

In [83]:
zp_data = cus_data[cus_data['cop_pref_o_ext_s_id'].isin(np.sort(data['cop_pref_o_ext_s_id'].unique())[:25])]

In [84]:
# Zooplus sites
np.sort(zp_data['cop_pref_o_ext_s_id'].unique())

array([ 1,  2,  3,  4,  7,  8, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22, 23,
       24, 25, 26, 28, 29, 30, 31, 32], dtype=int64)

In [85]:
# Only customers having 2 or more orders finished
zp_data = zp_data[zp_data['cop_g_order_count'] >= 2]

In [86]:
zp_data['OLD_TIER'].unique(), zp_data['NEW_TIER'].unique(), zp_data['cop_g_order_count'].min()

(array([   0.,    4.,    5.,   25., 1004., 1000.,   24.,    6., 1005.,
          nan]),
 array([   0.,    4.,    5.,   25., 1004.,   24., 1000., 1005.,    6.]),
 2)

In [87]:
zp_data = zp_data.dropna()

In [88]:
zp_data.sample(10)

,K_ID,OLD_TIER,OLD_BOX,NEW_TIER,NEW_BOX,K_REGISTRATION_S_ID,cop_c_id,cop_pref_o_ext_s_id,cop_g_order_count,cop_g_net_total,cop_g_cm2,CM2%
5900574,12935876,0.0,Z,4.0,A,12,12935876,12,4,175.319643,19.707991,11.241177
5410608,3108566,0.0,Z,0.0,Z,4,3108566,4,10,509.916666,83.973497,16.468082
1392574,14824507,0.0,Z,0.0,Z,21,14824507,21,3,243.069584,-7.638696,-3.142596
571381,2107244,0.0,Z,24.0,A,1,2107244,1,17,1045.904759,108.790702,10.401588
3661146,14126169,0.0,Z,0.0,Z,8,14126169,8,2,47.678614,0.729614,1.530275
7108081,13669076,0.0,Z,4.0,B4,4,13669076,4,6,250.933333,19.651497,7.831362
6228019,18487622,5.0,A,5.0,A,12,18487622,12,3,135.631148,35.510780,26.181877
1775536,15958518,0.0,Z,0.0,Z,8,15958518,8,2,42.830140,-3.485860,-8.138801
1661406,15593193,0.0,Z,0.0,Z,3,15593193,3,11,564.409870,20.721331,3.671327
1639823,15571154,4.0,B3,5.0,B4,7,15571154,7,5,357.801653,61.860633,17.289085


In [89]:
zp_crosstab = zp_data.fillna('not_segmented').groupby(['OLD_TIER', 'NEW_TIER'])['cop_c_id'].count().unstack()

In [90]:
zp_crosstab.fillna(0).astype(int)

NEW_TIER,0.0,4.0,5.0,6.0,24.0,25.0,1000.0,1004.0,1005.0
OLD_TIER,,,,,,,,,
0.0,1935809,575682,18485,103,13662,496,4397,15782,68
4.0,92034,971420,448755,120,242,63,8629,5453,943
5.0,13804,116847,523282,58,15,229,974,754,524
6.0,2984,2930,2224,400,69,28,42,38,6
24.0,1938,117,45,2,21628,6493,0,0,0
25.0,329,18,63,1,1709,7815,0,0,0
1000.0,2154,964,226,7,0,0,9368,0,0
1004.0,1448,257,35,0,0,0,0,8274,625
1005.0,357,66,79,0,0,0,0,1781,1706


In [91]:
zp_data = zp_data[(zp_data['OLD_TIER'] != float(6)) & (zp_data['NEW_TIER'] != float(6))]

In [92]:
zp_crosstab = zp_data.fillna('not_segmented').groupby(['OLD_TIER', 'NEW_TIER'])['cop_c_id'].count().unstack()

In [93]:
zp_crosstab.fillna(0).astype(int)

NEW_TIER,0.0,4.0,5.0,24.0,25.0,1000.0,1004.0,1005.0
OLD_TIER,,,,,,,,
0.0,1935809,575682,18485,13662,496,4397,15782,68
4.0,92034,971420,448755,242,63,8629,5453,943
5.0,13804,116847,523282,15,229,974,754,524
24.0,1938,117,45,21628,6493,0,0,0
25.0,329,18,63,1709,7815,0,0,0
1000.0,2154,964,226,0,0,9368,0,0
1004.0,1448,257,35,0,0,0,8274,625
1005.0,357,66,79,0,0,0,1781,1706


In [94]:
zp_crosstab['SUM_OLD_TIER'] = zp_crosstab.apply(np.sum, axis=1)

In [95]:
zp_crosstab.loc['SUM_NEW_TIER'] = zp_crosstab.apply(np.sum, axis=0)

In [96]:
zp_crosstab['OLD_DIST%'] = zp_crosstab['SUM_OLD_TIER'] / zp_crosstab['SUM_OLD_TIER']['SUM_NEW_TIER'] * 100

In [97]:
zp_crosstab.loc['NEW_DIST%'] = zp_crosstab.loc['SUM_NEW_TIER'] / zp_crosstab.loc['SUM_NEW_TIER']['SUM_OLD_TIER'] * 100

In [98]:
zp_crosstab.fillna(0).astype(int)

NEW_TIER,0.0,4.0,5.0,24.0,25.0,1000.0,1004.0,1005.0,SUM_OLD_TIER,OLD_DIST%
OLD_TIER,,,,,,,,,,
0.0,1935809,575682,18485,13662,496,4397,15782,68,2564381,53
4.0,92034,971420,448755,242,63,8629,5453,943,1527539,31
5.0,13804,116847,523282,15,229,974,754,524,656429,13
24.0,1938,117,45,21628,6493,0,0,0,30221,0
25.0,329,18,63,1709,7815,0,0,0,9934,0
1000.0,2154,964,226,0,0,9368,0,0,12712,0
1004.0,1448,257,35,0,0,0,8274,625,10639,0
1005.0,357,66,79,0,0,0,1781,1706,3989,0
SUM_NEW_TIER,2047873,1665371,990970,37256,15096,23368,32044,3866,4815844,100


In [99]:
zp_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4815844 entries, 2 to 9245370
Data columns (total 12 columns):
K_ID                   int64
OLD_TIER               float64
OLD_BOX                object
NEW_TIER               float64
NEW_BOX                object
K_REGISTRATION_S_ID    int64
cop_c_id               int64
cop_pref_o_ext_s_id    int64
cop_g_order_count      int64
cop_g_net_total        float64
cop_g_cm2              float64
CM2%                   float64
dtypes: float64(5), int64(5), object(2)
memory usage: 477.6+ MB


In [100]:
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: yellow' if v else '' for v in is_max]

In [101]:
for i in np.sort(zp_data['cop_pref_o_ext_s_id'].unique()):
    print('                                                SITE ' + str(i) + '                                                           ')
    cus_data = zp_data[zp_data['K_REGISTRATION_S_ID'] == i]
    crosstab = cus_data.fillna('not_segmented').groupby(['OLD_TIER', 'NEW_TIER'])['cop_c_id'].count().unstack()
    crosstab_rel = crosstab.apply(lambda x:x/x.sum() * 100, axis=1).fillna(0).round(2)
    display(crosstab_rel.style.apply(highlight_max, axis=1))
    crosstab['SUM_OLD_TIER'] = crosstab.apply(np.sum, axis=1)
    crosstab.loc['SUM_NEW_TIER'] = crosstab.apply(np.sum, axis=0)
    crosstab['OLD_DIST%'] = crosstab['SUM_OLD_TIER'] / crosstab['SUM_OLD_TIER']['SUM_NEW_TIER'] * 100
    crosstab.loc['NEW_DIST%'] = crosstab.loc['SUM_NEW_TIER'] / crosstab.loc['SUM_NEW_TIER']['SUM_OLD_TIER'] * 100
    display(crosstab.fillna(0).astype(int))
    print('Percentage of non-eligible customers with the old routine: ',
          np.round(cus_data[(cus_data['OLD_TIER'] == 0) | (cus_data['OLD_TIER'].isnull())]['cop_c_id'].count() / cus_data['cop_c_id'].count() * 100, 2))
    print('Percentage of non-eligible customers with the new routine: ',
          np.round(cus_data[(cus_data['NEW_TIER'] == 0) | (cus_data['NEW_TIER'].isnull())]['cop_c_id'].count() / cus_data['cop_c_id'].count() * 100, 2))
    print('_______________________________________________________________________________________________________________________')
    print('')

                                                SITE 1                                                           


NEW_TIER,0.0,4.0,5.0,24.0,25.0,1000.0,1004.0,1005.0
OLD_TIER,,,,,,,,
0.0,74.81,21.79,0.88,2.42,0.09,0.01,0,0
4.0,4.75,68.38,26.79,0.06,0.01,0.01,0,0
5.0,2.09,22.18,75.61,0.01,0.11,0,0,0
24.0,6.4,0.31,0.12,71.73,21.45,0,0,0
25.0,3.33,0.15,0.49,17.27,78.76,0,0,0
1000.0,12,8,4,0,0,76,0,0
1004.0,13.04,0,0,0,0,0,69.57,17.39
1005.0,0,0,0,0,0,0,85.71,14.29


NEW_TIER,0.0,4.0,5.0,24.0,25.0,1000.0,1004.0,1005.0,SUM_OLD_TIER,OLD_DIST%
OLD_TIER,,,,,,,,,,
0.0,421557,122786,4983,13626,494,32,25,0,563503,46
4.0,19106,274953,107710,232,58,30,19,3,402111,33
5.0,4314,45772,156010,14,228,4,2,2,206346,17
24.0,1922,92,36,21552,6446,0,0,0,30048,2
25.0,327,15,48,1698,7742,0,0,0,9830,0
1000.0,6,4,2,0,0,38,0,0,50,0
1004.0,3,0,0,0,0,0,16,4,23,0
1005.0,0,0,0,0,0,0,6,1,7,0
SUM_NEW_TIER,447235,443622,268789,37122,14968,104,68,10,1211918,100


Percentage of non-eligible customers with the old routine:  46.5
Percentage of non-eligible customers with the new routine:  36.9
_______________________________________________________________________________________________________________________

                                                SITE 2                                                           


NEW_TIER,0.0,4.0,5.0,24.0,25.0,1000.0,1004.0,1005.0
OLD_TIER,,,,,,,,
0.0,84.62,14.35,0.8,0.07,0,0.02,0.14,0
4.0,8.38,63.14,28.18,0.04,0.04,0.19,0.04,0
5.0,3.23,21.69,75,0,0,0,0.08,0
24.0,4.76,0,0,52.38,42.86,0,0,0
25.0,4.76,0,4.76,4.76,85.71,0,0,0
1000.0,0,16.67,16.67,0,0,66.67,0,0
1004.0,50,0,0,0,0,0,50,0
1005.0,0,0,0,0,0,0,33.33,66.67


...

NEW_TIER,0.0,4.0,5.0,SUM_OLD_TIER,OLD_DIST%
OLD_TIER,,,,,
0.0,6664,164,5,6833,52
4.0,310,3669,304,4283,33
5.0,32,115,1683,1830,14
SUM_NEW_TIER,7006,3948,1992,12946,100
NEW_DIST%,54,30,15,100,0


Percentage of non-eligible customers with the old routine:  52.78
Percentage of non-eligible customers with the new routine:  54.12
_______________________________________________________________________________________________________________________

                                                SITE 32                                                           


NEW_TIER,0.0,4.0,5.0
OLD_TIER,,,
0.0,99.21,0.79,0
4.0,35.42,60.62,3.96
5.0,0,28.36,71.64


NEW_TIER,0.0,4.0,5.0,SUM_OLD_TIER,OLD_DIST%
OLD_TIER,,,,,
0.0,2642,21,0,2663,81
4.0,170,291,19,480,14
5.0,0,38,96,134,4
SUM_NEW_TIER,2812,350,115,3277,100
NEW_DIST%,85,10,3,100,3


Percentage of non-eligible customers with the old routine:  81.26
Percentage of non-eligible customers with the new routine:  85.81
_______________________________________________________________________________________________________________________

